### Init

In [235]:
import robotic as ry
import time

try:
    del C
    del bot
except:
    pass

C = ry.Config()
# C.watchFile("throwing1.g")
C.addFile("throwing1.g")
qHome = C.getJointState()
C.view()
#time.sleep(30)
C.getFrame('cargo').unLink()
C.view()

0

### Komo probs

In [236]:
way0 = C.addFrame('way0', 'start_area')

way0.setShape(ry.ST.marker, size=[.1])
way0.setRelativePose('t(0.2 0.15 0.1) d(90 0 0 0)')

way1 = C.addFrame('way1', 'start_area')

way1.setShape(ry.ST.marker, size=[.2])
way1.setPosition(C.getFrame('l_gripper').getPosition())

C.view()

0

In [237]:
def pre_grasp_komo(C, gripper_name, grasp_frame_name, q0, qHome):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0) 
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome)

    komo.addObjective([1., 3.], ry.FS.positionRel, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [-0.08, 0, 0])  

    # komo.addObjective([1., 3.], ry.FS.scalarProductXX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 
    komo.addObjective([1., 3.], ry.FS.scalarProductYY, [gripper_name, grasp_frame_name], ry.OT.eq, [1e2], [1]) 
    # komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 

    komo.addObjective([1., 3.], ry.FS.scalarProductXZ, [grasp_frame_name, gripper_name], ry.OT.eq, [1e1], [-1]) 


    ret = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret)
    return komo

In [238]:
def create_grasp_komo(C, gripper_name, grasp_frame_name, q0, qHome):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e-1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0) 
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome)

    komo.addObjective([1., 3.], ry.FS.positionRel, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [-0.08, 0, 0])  

    # komo.addObjective([1., 3.], ry.FS.scalarProductXX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 
    komo.addObjective([1., 3.], ry.FS.scalarProductYY, [gripper_name, grasp_frame_name], ry.OT.eq, [1e2], [1]) 
    # komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 

    komo.addObjective([1., 3.], ry.FS.scalarProductXZ, [grasp_frame_name, gripper_name], ry.OT.eq, [1e1], [-1]) 


    ret = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret)
    return komo

In [239]:
def throw_position_komo(C, gripper_name, grasp_frame_name):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([1.], ry.FS.positionDiff, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0, 0, 0])
    komo.addObjective([3.], ry.FS.scalarProductXX, [gripper_name, 'way0'], ry.OT.eq, [1e1], [0])
    ret2 = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret2)
    return komo

In [240]:
def post_throw_position_komo(C, gripper_name, throw_waypoint_frame):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([1.], ry.FS.positionDiff, [gripper_name, throw_waypoint_frame], ry.OT.eq, [1e1], [0, 0, 0])
    komo.addObjective([3.], ry.FS.scalarProductXX, [gripper_name, 'way1'], ry.OT.eq, [1e1], [0])
    ret3 = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret3)
    return komo

In [241]:
q0 = C.getJointState()
komo0 = pre_grasp_komo(C, 'l_gripper', 'paddle_pick_point', q0, qHome)
path0 = komo0.getPath()

komo = create_grasp_komo(C, 'l_gripper', 'paddle', q0, qHome)
path = komo.getPath()

komo2 = throw_position_komo(C, 'l_gripper', 'cargo')
path2 = komo2.getPath()

komo3 = post_throw_position_komo(C, 'l_gripper', 'way1')
path3 = komo3.getPath()

{ time: 0.034076, evals: 179, done: 1, feasible: 1, sos: 0.87113, f: 0, ineq: 0, eq: 0.00170887 }
{ time: 0.06128, evals: 283, done: 1, feasible: 1, sos: 0.915599, f: 0, ineq: 0, eq: 0.00584472 }
{ time: 0.000578, evals: 5, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.00269333 }
{ time: 0.000357, evals: 4, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.0175228 }


### Pick up ball

In [242]:
bot = ry.BotOp(C, useRealRobot=False)
bot.home(C)
# print(path.shape)
bot.move(path0, [1.])
bot.move(path, [3.])
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)

bot.gripperCloseGrasp(ry._left, 'paddle') 
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

C.getFrame('paddle').unLink()
C.attach('l_gripper', 'paddle')

print(path2.shape)
bot.move(path2, [3.])
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)

# pre_throw_pos = C.getFrame('l_gripper').getPosition()

# bot.move(path3, [3.])
# bot.gripperMove(ry._left, width=0.075, speed=5)
# while bot.getTimeToEnd() > 0:
#     bot.sync(C, .1)


# bot.home(C)

(3, 7)


### Calculate throw

In [243]:
import numpy as np
from scipy.optimize import minimize

def calculate_throw_parameters_bin(initial_position, bin_dimensions, gravity=9.81):
    def loss_function(params):
        release_height, v_magnitude, theta_xz = params
        x0, y0, z0 = initial_position
        bottom_left = bin_dimensions['bottom_left']
        top_right = bin_dimensions['top_right']
        
        release_position = (x0, y0, release_height)
        
        v_x = v_magnitude * np.cos(theta_xz)
        v_z = v_magnitude * np.sin(theta_xz)
        
        t_flight = 2 * v_z / gravity 
        
        x_landing = x0 + v_x * t_flight
        
        within_x_bounds = bottom_left[0] <= x_landing <= top_right[0]
        within_y_bounds = bottom_left[1] <= y0 <= top_right[1]
        
        if within_x_bounds and within_y_bounds:
            return 0  
        else:
            dx = max(0, bottom_left[0] - x_landing, x_landing - top_right[0])
            dy = max(0, bottom_left[1] - y0, y0 - top_right[1])
            return dx + dy 

    initial_guess = [initial_position[2], 5.0, np.pi / 4] 
    
    bounds = [(initial_position[2], initial_position[2] + 1.0), 
              (1.0, 20.0),  
              (0, np.pi / 2)]  
    
    result = minimize(loss_function, initial_guess, bounds=bounds, method='SLSQP')
    
    if result.success:
        release_height, v_magnitude, theta_xz = result.x
        v_x = v_magnitude * np.cos(theta_xz)
        v_z = v_magnitude * np.sin(theta_xz)
        return {
            'release_position': (initial_position[0], initial_position[1], release_height),
            'velocity': (v_x, 0, v_z),
            'angle_xz': theta_xz,
        }
    else:
        raise ValueError("Optimization failed to find valid parameters.")

initial_pos = C.getFrame('l_gripper').getPosition()
print("Initial Position:", initial_pos)
bin_dims = {
    'bottom_left': (-0.25, -0.75, 0.14), 
    'top_right': (0.25, -0.25, 0.24)  
}
# result = calculate_throw_parameters_bin(initial_pos, bin_dims)
result = calculate_throw_parameters_bin(pre_throw_pos, bin_dims)

print("Release Position:", result['release_position'])
print("Velocity:", result['velocity'])
print("Release Angle (radians):", result['angle_xz'])


Initial Position: [0.27944538 1.81916166 0.98945727]
Release Position: (0.28321953469759653, 1.8338805389457, 0.9990845584890502)
Velocity: (0.7071055813490473, 0, 0.7071079810220118)
Release Angle (radians): 0.7853998602224741


In [244]:
# Add frame for the release position
release_frame = C.addFrame('release')
release_frame.setShape(ry.ST.marker, size=[0])
release_frame.setPosition(result['release_position'])
C.view()

0

### Clean up

In [245]:
del C
del bot